# ELASTIC BRUSSELS NETWORK STA

This file first shows how to run an elastic Static Traffic Assignment in dyntapy on a larger network. It also shows examples of how to run an elastic assignment with tolls.

1. At first, you import all the necessary packages and files from mostly dyntapy. 

In [ ]:
import warnings
warnings.filterwarnings('ignore') # hide warnings
import os
import sys
sys.path.append("../../..")
import numpy as np
import pandas as pd
import geopandas as gpd
from dyntapy.supply_data import road_network_from_place, relabel_graph
from dyntapy.demand_data import add_centroids, od_graph_from_matrix
from dyntapy.assignments import StaticAssignment
from dyntapy.visualization import show_network
from dyntapy.results import get_od_flows
from dyntapy.toll import create_toll_object
from pyproj import Proj, transform
from pickle import dump


2. In this case we would like to load a specific network. To make the path a bit more flexible, the buffer parameter which is the radius of the zoning file and the city are parameters that can be adapted easily to call a specific file path. Here a radius of 40km and the city of Brussels are used. These steps are executed by fully running the next block of code. 

In [ ]:
# Creating the network for Brussels: 
# fill in these parameters 
# IMPORTANT: Use the same parameter values for the buffer as was done in STA_prep_script!
buffer = 40
city = 'BRUSSEL'

HERE = os.path.dirname(os.path.realpath("__file__"))
data_path = HERE + os.path.sep + os.pardir + os.path.sep + os.pardir + os.path.sep + 'data_map'

zoning_path = data_path  + os.path.sep + 'QGIS' + os.path.sep + city + "_" + str(buffer) + "_10_aggr_comb.shp"
od_path = data_path + os.path.sep + 'STA' + os.path.sep + 'OD_matrix' + os.path.sep + city + "_" + str(buffer) + "_9_aggregated.xlsx"

3. The next steps are always the same: 

- Create or retrieve a network
- Create and add centroids to network → adds connectors automatically
- Create or load an OD matrix (with the demand) onto the centroids

  One of the important steps that you should not forget is projecting everything in the correct coordinate reference system (CRS). Especially when zones are aggregated using QGIS, this is something that you should be aware of. 

In [ ]:
# Creating the network for Brussels. 
network = road_network_from_place("Brussels", buffer_dist_close=buffer*1000)
network = relabel_graph(network)
# show_network(network,notebook=True)

zoning = gpd.read_file(zoning_path)
od = pd.read_excel(od_path)
old_od = od.to_numpy() # The OD matrix is now stored in a numpy array

# Retrieve zone number, x_centroid (LON) and y_centroid (LAT) from each zone. 
zone_numbers = zoning["ZONENUMMER"]
x_lamb = zoning["X_LAMB"]
x_lamb = x_lamb.to_numpy()
y_lamb = zoning["Y_LAMB"]
y_lamb = y_lamb.to_numpy()

# Project to correct CRS. 
inProj = Proj(init='epsg:31370')
outProj = Proj(init='epsg:4326')
x_centroids, y_centroids = transform(inProj,outProj,x_lamb,y_lamb)

# Add the centroids to the network. Relabelling the graph is required (see demo for reason why)
network = add_centroids(network, x_centroids,y_centroids,k=1, method='link')
network = relabel_graph(network)
# show_network(network, notebook=True)

# Create OD graph and plot demand
old_graph = od_graph_from_matrix(old_od,x_centroids,y_centroids) 

4. To have an elastic assignment, the inverse demand function approach is used. To calculate the A matrix, first an STA without toll needs to be run to get a price-demand relation. This can be done using the already familiar piece of code running the assignment below. The demand that is put on the network will in the first place remain static, being the original OD graph. 

   To determine A, you should have three other elements: B (elasticity matrix from literature) and a related price-demand value as P and D. These last ones can be extracted from the previous untolled assignment. The demand D is known and the impedance or P between each origin-destination pair can be calculated via the built-in function to extract the skims. Afterwards you just fill in the inverse demand function to get a value for A.

In [ ]:
# 1. STA without toll
assignment = StaticAssignment(network,old_graph)
result = assignment.run('dial_b')

# 2. Load B matrix (elasticities) 
elasticity_path = data_path + os.path.sep + 'STA' + os.path.sep + 'elasticity' + os.path.sep + 'Brussel_40'
B = np.loadtxt(elasticity_path)

# 3. Calculate A based on B, D and P
A = old_od*B + result.skim

    Congratulations, you now have every element to start the loop to determine realistic demand via elasticities!

Now that the parameters have been tuned/estimated, we can start running STAs with tolls. For this, we do a loop of STAs (with a different OD-matrix in every iteration) until the elastic demand converges for the given toll. With this converged demand we can finally do the actual STA and compute the objective value.

In [ ]:
# New assignment with toll to calculate new OD matrix 
toll_value = 0.05
toll_ids = [1490]
toll_method = 'single'
toll = create_toll_object(network, toll_method, toll_ids, toll_value)

assignment = StaticAssignment(network,old_graph,toll)
result = assignment.run('dial_b')

# use A and B with second assignment to determine new_OD (necessary to enter while loop)
new_od = (A-result.skim)/B
print(abs((np.linalg.norm(new_od) - np.linalg.norm(old_od))/np.linalg.norm(old_od)))

In [ ]:
# function that modifies one random value of each zero-row/zero-column to a non-zero value
def modify_zeroes_in_od(od, value):
    indices_zero_rows = np.where((od==0).all(axis=1))[0]
    indices_zero_cols = np.where((od==0).all(axis=0))[0]
    for i in indices_zero_rows:
        j = np.random.choice(od.shape[1])
        od[i,j] = value

    for j in indices_zero_cols:
        i = np.random.choice(od.shape[1])
        od[i,j] = value
    return od

new_od = modify_zeroes_in_od(new_od, 0.0001)


In [ ]:
i = 1 
max_iterations = 3

while abs((np.linalg.norm(new_od) - np.linalg.norm(old_od))/np.linalg.norm(old_od)) > 0.05 and i < max_iterations:
    print("started iteration:", i)
    old_od = new_od
    old_graph =  od_graph_from_matrix(old_od, x_centroids, y_centroids)
    assignment = StaticAssignment(network,old_graph, toll)
    result = assignment.run('dial_b')
    new_od = old_od + ((A-result.skim)/B - old_od)/i
    new_od = modify_zeroes_in_od(new_od, 0.0001)
    i += 1
    print("Frobenius norm deviation: ", abs((np.linalg.norm(new_od) - np.linalg.norm(old_od))/np.linalg.norm(old_od)))

In [ ]:
# Compute objective for last OD-matrix on which we ran an STA. 
veh_hours = result.link_costs * result.flows
objective = sum(veh_hours)

# 2. HEEDS preparation

In [ ]:
buffer = str(buffer)
HEEDS_path = HERE + os.path.sep + os.pardir + os.path.sep + os.pardir + os.path.sep + 'data_map' + os.path.sep + 'HEEDS_input'
network_path = HEEDS_path + os.path.sep + 'network_with_centroids' + os.path.sep + 'elastic_' + city + '_' + buffer
od_matrix_path = HEEDS_path + os.path.sep + 'od_graph' + os.path.sep + 'elastic_' + city + '_' + buffer + '.xlsx'
A_matrix_path = HEEDS_path + os.path.sep + 'elastic' + os.path.sep + 'A_matrix_' + city + '_' + buffer
B_matrix_path = HEEDS_path + os.path.sep + 'elastic' + os.path.sep + 'B_matrix_' + city + '_' + buffer
x_centroids_path = HEEDS_path + os.path.sep + 'elastic' + os.path.sep + 'x_centroids_' + city + '_' + buffer
y_centroids_path = HEEDS_path + os.path.sep + 'elastic' + os.path.sep + 'y_centroids_' + city + '_' + buffer

with open(network_path, 'wb') as network_file:
    dump(network, network_file)
    print(f'network saved at f{network_path}')

od.to_excel(od_matrix_path, index=False)

with open(A_matrix_path, 'wb') as matrix:
    dump(A, matrix)
    print(f'A_matrix saved at f{A_matrix_path}')

with open(B_matrix_path, 'wb') as matrix:
    dump(B, matrix)
    print(f'B_matrix saved at f{B_matrix_path}')

np.savetxt(x_centroids_path,x_centroids)
np.savetxt(y_centroids_path, y_centroids)


